In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

# EmbededTraversal(cloned::struct,obj::struct,n::int,suffix::string)

return the cloned obj of struct type who consist in a struct with the same body as the struct obj passed as arg.every element of the body is visited and copied in the struct cloned using local variable to support, the element in the body can have diffent type :tuple,array or other struct

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "8447826383766830820", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "16990717297481722407", 2, "feature")

In [6]:


function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	for i=1:length(obj.body)
		if isa(obj.body[i],Matrix)
			mat = obj.body[i]
			d,d = size(mat)
			newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
			for h in range(1, length=d)
				for k in range(1, length=d)
					newMat[h,k]=mat[h,k]
				end
			end
			push!(cloned.body,[newMat])
		elseif (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==3
			V,FV,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,FV,EV)])
		elseif  (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==2
			V,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,EV)])
		elseif isa(obj.body[i],Lar.Struct)
			newObj = Struct()
			newObj.box = [ [obj.body[i].box[1];zeros(dimadd)],
							[obj.body[i].box[2];zeros(dimadd)] ]
			newObj.category = (obj.body[i]).category
			push!(cloned.body,[embedTraversal(newObj,obj.body[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end



embedTraversal (generic function with 1 method)

In [7]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  2.433 μs (25 allocations: 1.81 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "3805851539538212783", Any, "feature")

In [8]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  19.200 μs (87 allocations: 4.61 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "4469424143845526420", Any, "feature")